In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import QuantLib as ql
import datetime

In [3]:
# set u quantlib
ql.Settings.instance().evaluationDate = ql.Date(1,2,2013)
calendar = ql.UnitedStates()

In [4]:
def convert_to_quantlib_datetime(df, date_columns):
    """
    Convert specified columns in a dataframe to QuantLib DateTime types.

    Parameters:
    - df: pandas DataFrame containing the data.
    - date_columns: list of column names to be converted.

    Returns:
    - DataFrame with specified columns converted to QuantLib DateTime types.
    """
    for col in date_columns:
        # Convert pandas datetime to QuantLib Date
        col2 = col + '_ql'
        df[col2] = df[col].apply(lambda x: ql.Date(x.day, x.month, x.year) if pd.notnull(x) else None)

    return df

In [5]:
data = pd.read_excel("data/USD_CAD_combined.xlsx")

In [6]:
# Specify the date columns to be converted
date_columns = ['File Date', 'Effective', 'Maturity', 'Trade Date']

data["File Date"] = pd.to_datetime(data["File Date"])
data["Trade Date"] = pd.to_datetime(data["Trade Date"])
data["Effective"] = pd.to_datetime(data["Effective"])
data["Maturity"] = pd.to_datetime(data["Maturity"])

data = convert_to_quantlib_datetime(df=data, date_columns=date_columns)


In [7]:

data["Days Between Trade and Effective"] = data.apply(lambda x: calendar.businessDaysBetween(x["Trade Date_ql"], x["Effective_ql"], includeFirst=False, includeLast=True), axis=1)

In [8]:
usd = data[data["Curr"] == "USD"].groupby(("Days Between Trade and Effective")).count()
cad = data[data["Curr"] == "CAD"].groupby(("Days Between Trade and Effective")).count()

In [9]:
# Filter by CD, Leg 1, Rate 2, Othr Pmnt, and T
filtered_data = data[(data['CD'] == "TR") & 
                     (data['Leg 1'].str.upper() == "FIXED") & 
                     (data['Rate 2'].isnull()) & 
                     (data['Othr Pmnt'].isnull()) & 
                     (data['T'].isin(["1Y", "2Y", "3Y", "4Y", "5Y", "6Y", "7Y", "8Y", "9Y", "10Y"]))]


# For USD currency, Leg 2 should be "USD-LIBOR-BBA"
# For CAD currency, Leg 2 should be "CAD-BA-CDOR"
filtered_data = filtered_data[((filtered_data['Curr'] == "USD") & (filtered_data['Leg 2'] == "USD-LIBOR-BBA") & (filtered_data["Days Between Trade and Effective"] == 2)
                               & (filtered_data["PF 1"] == "3M") & (filtered_data["PF 2"] == "6M")) |
                             ((filtered_data['Curr'] == "CAD") & (filtered_data['Leg 2'] == "CAD-BA-CDOR") & (filtered_data["Days Between Trade and Effective"] == 0) \
                              & (filtered_data["PF 1"] == "6M") & (filtered_data["PF 2"] == "6M"))]
filtered_data["T"] = filtered_data.apply(lambda x: round((x["Maturity_ql"] - x["Effective_ql"])/365.25), axis=1)

filtered_data.drop(["File Date_ql", "Effective_ql", "Maturity_ql", "Trade Date_ql"], axis=1, inplace=True)

filtered_data.describe()


KeyError: 'Effecttive_ql'

In [ ]:
filtered_data.to_excel("data/filtered_data.xlsx")